In [57]:
import pandas as pd
import numpy as np
import nltk
import os
from ast import literal_eval
import string
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
import pickle

In [58]:
os.getcwd()

'f:\\ML projects\\MOVIE-RECEOMMENDATION'

In [59]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [60]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [61]:
movies.shape

(4803, 20)

In [62]:
movies=movies[movies['vote_count']>1000]

In [63]:
movies.shape

(921, 20)

In [64]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

# step1: getting the dataset ready

features that we might actually need!

genres, id, keywords, overview, production_companies, title

In [65]:
data=movies[['id','title', 'genres', 'overview', 'keywords', 'production_companies']]

In [66]:
data=data.merge(credits, on='title')
data.drop(['movie_id'], axis=1, inplace=True)
data.head()

,id,title,genres,overview,keywords,production_companies,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","Captain Barbossa, long believed to be dead, ha...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",A cryptic message from Bond’s past sends him o...,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",Following the death of District Attorney Harve...,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","John Carter is a war-weary, former military ca...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


# step2: preprocessing


our goal is to combine all these features ('genres', 'overview', 'keywords', 'production_companies') into a single column 

and these cols ('genres',  'keywords', 'production_companies') are in a weird format, list of dics inside a string

In [67]:
data.genres[0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [68]:
def get_genre(text):
    L=[]
    for i in literal_eval(text):
        L.append(i['name'])
    return L

data.genres=data.genres.apply(get_genre)

In [69]:
data.head(2)

,id,title,genres,overview,keywords,production_companies,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","Captain Barbossa, long believed to be dead, ha...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [70]:
data.production_companies[0]

'[{"name": "Ingenious Film Partners", "id": 289}, {"name": "Twentieth Century Fox Film Corporation", "id": 306}, {"name": "Dune Entertainment", "id": 444}, {"name": "Lightstorm Entertainment", "id": 574}]'

In [71]:
def get_productionComapnies(text):
    L=[]
    for i in literal_eval(text):
        L.append(i['name'])
    return L

data.production_companies=data.production_companies.apply(get_productionComapnies)

data.head(1)

,id,title,genres,overview,keywords,production_companies,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[Ingenious Film Partners, Twentieth Century Fo...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [72]:
def get_keywords(text):
    L=[]
    for i in literal_eval(text):
        L.append(i['name'])
    return L

data.keywords=data.keywords.apply(get_keywords)

data.head(1)

,id,title,genres,overview,keywords,production_companies,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Ingenious Film Partners, Twentieth Century Fo...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [73]:
# here we will only take top 3 actors/actresses

def get_cast(text):
    L=[]
    c=0
    for i in literal_eval(text):
        if c!=3:
            L.append(i['name'])
            c+=1
        else:
            break
    return L

data.cast=data.cast.apply(get_cast)

In [74]:
data.head()

,id,title,genres,overview,keywords,production_companies,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Ingenious Film Partners, Twentieth Century Fo...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","Captain Barbossa, long believed to be dead, ha...","[ocean, drug abuse, exotic island, east india ...","[Walt Disney Pictures, Jerry Bruckheimer Films...","[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[Action, Adventure, Crime]",A cryptic message from Bond’s past sends him o...,"[spy, based on novel, secret agent, sequel, mi...","[Columbia Pictures, Danjaq, B24]","[Daniel Craig, Christoph Waltz, Léa Seydoux]","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",Following the death of District Attorney Harve...,"[dc comics, crime fighter, terrorist, secret i...","[Legendary Pictures, Warner Bros., DC Entertai...","[Christian Bale, Michael Caine, Gary Oldman]","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[Action, Adventure, Science Fiction]","John Carter is a war-weary, former military ca...","[based on novel, mars, medallion, space travel...",[Walt Disney Pictures],"[Taylor Kitsch, Lynn Collins, Samantha Morton]","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [75]:
# with crew, we are only accepting directors name
def get_crew(text):
    L=[]
    for i in literal_eval(text):
        if i['job']=='Director':
            L.append(i['name'])
    return L

data.crew=data.crew.apply(get_crew)

In [76]:
data.head()

,id,title,genres,overview,keywords,production_companies,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Ingenious Film Partners, Twentieth Century Fo...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","Captain Barbossa, long believed to be dead, ha...","[ocean, drug abuse, exotic island, east india ...","[Walt Disney Pictures, Jerry Bruckheimer Films...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[Action, Adventure, Crime]",A cryptic message from Bond’s past sends him o...,"[spy, based on novel, secret agent, sequel, mi...","[Columbia Pictures, Danjaq, B24]","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",Following the death of District Attorney Harve...,"[dc comics, crime fighter, terrorist, secret i...","[Legendary Pictures, Warner Bros., DC Entertai...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[Action, Adventure, Science Fiction]","John Carter is a war-weary, former military ca...","[based on novel, mars, medallion, space travel...",[Walt Disney Pictures],"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [77]:
data=data.dropna(how='any')

In [78]:
data['overview']=data['overview'].apply(lambda x:x.split())

In [79]:
data.head()

,id,title,genres,overview,keywords,production_companies,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[In, the, 22nd, century,, a, paraplegic, Marin...","[culture clash, future, space war, space colon...","[Ingenious Film Partners, Twentieth Century Fo...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[Captain, Barbossa,, long, believed, to, be, d...","[ocean, drug abuse, exotic island, east india ...","[Walt Disney Pictures, Jerry Bruckheimer Films...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[Action, Adventure, Crime]","[A, cryptic, message, from, Bond’s, past, send...","[spy, based on novel, secret agent, sequel, mi...","[Columbia Pictures, Danjaq, B24]","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[Following, the, death, of, District, Attorney...","[dc comics, crime fighter, terrorist, secret i...","[Legendary Pictures, Warner Bros., DC Entertai...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[Action, Adventure, Science Fiction]","[John, Carter, is, a, war-weary,, former, mili...","[based on novel, mars, medallion, space travel...",[Walt Disney Pictures],"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [80]:
data['genres']=data['genres'].apply(lambda x: [i.replace(" ", '') for i in x])
data['keywords']=data['keywords'].apply(lambda x: [i.replace(" ", '') for i in x])
data['cast']=data['cast'].apply(lambda x: [i.replace(" ", '') for i in x])
data['crew']=data['crew'].apply(lambda x: [i.replace(" ", '') for i in x])
data['production_companies']=data['production_companies'].apply(lambda x: [i.replace(" ", '') for i in x])

In [81]:
data.head()

,id,title,genres,overview,keywords,production_companies,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[In, the, 22nd, century,, a, paraplegic, Marin...","[cultureclash, future, spacewar, spacecolony, ...","[IngeniousFilmPartners, TwentiethCenturyFoxFil...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[Captain, Barbossa,, long, believed, to, be, d...","[ocean, drugabuse, exoticisland, eastindiatrad...","[WaltDisneyPictures, JerryBruckheimerFilms, Se...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[Action, Adventure, Crime]","[A, cryptic, message, from, Bond’s, past, send...","[spy, basedonnovel, secretagent, sequel, mi6, ...","[ColumbiaPictures, Danjaq, B24]","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[Following, the, death, of, District, Attorney...","[dccomics, crimefighter, terrorist, secretiden...","[LegendaryPictures, WarnerBros., DCEntertainme...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[Action, Adventure, ScienceFiction]","[John, Carter, is, a, war-weary,, former, mili...","[basedonnovel, mars, medallion, spacetravel, p...",[WaltDisneyPictures],"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [82]:
data['summary']=data['genres'] + data['overview'] +data['production_companies']+ data['keywords'] + data['cast'] + data['crew']

In [83]:
df=data

In [84]:
df.columns

Index(['id', 'title', 'genres', 'overview', 'keywords', 'production_companies',
       'cast', 'crew', 'summary'],
      dtype='object')

In [85]:
df.drop(['genres', 'overview','production_companies', 'keywords', 'cast', 'crew'], axis=1, inplace=True)

In [86]:
df['summary']=df['summary'].apply(lambda x: ' '.join(x))

In [87]:
df.head()

,id,title,summary
0,19995,Avatar,Action Adventure Fantasy ScienceFiction In the...
1,285,Pirates of the Caribbean: At World's End,"Adventure Fantasy Action Captain Barbossa, lon..."
2,206647,Spectre,Action Adventure Crime A cryptic message from ...
3,49026,The Dark Knight Rises,Action Crime Drama Thriller Following the deat...
4,49529,John Carter,Action Adventure ScienceFiction John Carter is...


In [88]:
df.summary[1]

"Adventure Fantasy Action Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems. WaltDisneyPictures JerryBruckheimerFilms SecondMateProductions ocean drugabuse exoticisland eastindiatradingcompany loveofone'slife traitor shipwreck strongwoman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski"

now we have to:
1. lower the text
2. remove punctuations
3. remove stopwords

In [89]:
df['summary']=df['summary'].apply(lambda x: x.lower()) # lowering the summary

In [90]:
# df['summary']=df['summary'].apply(lambda x: re.sub(r'[^\w\s]', '', x)) # removing the punctuations

In [91]:
def convert(text): # removing the stopwords
    string=''
    words=word_tokenize(text)
    filtered_words=[word for word in words if word.lower() not in stopwords.words('english')]
    clean_text=' '.join(filtered_words)
    return clean_text
df['summary']=df['summary'].apply(convert)

In [92]:
# lemmatization 

lemma=WordNetLemmatizer() 

def lemmatization(text):
    words=word_tokenize(text)
    lemmatized_string = ' '.join([lemma.lemmatize(word) for word in words])
    return lemmatized_string

df['summary']=df['summary'].apply(lemmatization)

In [93]:
df.summary[1]

"adventure fantasy action captain barbossa , long believed dead , come back life headed edge earth turner elizabeth swann . nothing quite seems . waltdisneypictures jerrybruckheimerfilms secondmateproductions ocean drugabuse exoticisland eastindiatradingcompany loveofone'slife traitor shipwreck strongwoman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger johnnydepp orlandobloom keiraknightley goreverbinski"

In [94]:
df.title=df.title.apply(lambda x: x.lower())

In [95]:
df

,id,title,summary
0,19995,avatar,action adventure fantasy sciencefiction 22nd c...
1,285,pirates of the caribbean: at world's end,"adventure fantasy action captain barbossa , lo..."
2,206647,spectre,action adventure crime cryptic message bond ’ ...
3,49026,the dark knight rises,action crime drama thriller following death di...
4,49529,john carter,action adventure sciencefiction john carter wa...
...,...,...,...
918,23827,paranormal activity,"horror mystery young , middle class couple mov..."
919,762,monty python and the holy grail,"adventure comedy fantasy king arthur , accompa..."
920,389,12 angry men,drama defense prosecution rested jury filing j...
921,431,cube,thriller sciencefiction mystery seven stranger...


# step3: text vectorization
Using tf-idf

In [96]:
TF_IDFvectorizer = TfidfVectorizer(max_features=20000, norm='l2', ngram_range=(1,2), use_idf=True)
vectors = TF_IDFvectorizer.fit_transform(df.summary).toarray()

In [97]:
vectors.shape

(923, 20000)

In [98]:
vectors[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [99]:
TF_IDFvectorizer.get_feature_names_out()

array(['00', '000', '007', ..., 'zooeydeschanel', 'zookeeper',
       'zuckerberg'], dtype=object)

In [100]:
similarity=cosine_similarity(vectors)

In [101]:
similarity[1]

array([0.01388196, 1.        , 0.01107164, 0.00128191, 0.02056329,
       0.04905536, 0.00706914, 0.02611585, 0.01396431, 0.01564528,
       0.02403362, 0.00587062, 0.49122465, 0.05661242, 0.02384552,
       0.00836212, 0.0167684 , 0.13998274, 0.02387689, 0.02043119,
       0.01228283, 0.00428827, 0.01874276, 0.01621027, 0.04364974,
       0.04186245, 0.02024131, 0.01325861, 0.0036223 , 0.0035015 ,
       0.05001364, 0.01039539, 0.02319795, 0.0106536 , 0.01831876,
       0.0136214 , 0.00898704, 0.0140199 , 0.01899183, 0.01146596,
       0.0125066 , 0.00945718, 0.00703951, 0.00788489, 0.0019988 ,
       0.02163318, 0.02141713, 0.02013058, 0.01965885, 0.00989076,
       0.06434548, 0.00490422, 0.01867057, 0.00309544, 0.01316456,
       0.01428406, 0.00477776, 0.04065723, 0.01809435, 0.        ,
       0.0305855 , 0.01797028, 0.00489513, 0.0067837 , 0.0013377 ,
       0.02991514, 0.0042653 , 0.01221554, 0.01019823, 0.00384962,
       0.01768898, 0.01517092, 0.0040503 , 0.00766494, 0.00365

# step4: building the recommendation function

In [102]:
def recommend(text):
    text=text.lower()
    index=df[df['title']==text].index[0]
    distances=similarity[index]
    movie_list=sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[:10]
    
    for i in movie_list:
        print(df.iloc[i[0]].title.title())

In [103]:
recommend('''avatar''')

Avatar
Aliens
Star Trek Into Darkness
Battle: Los Angeles
Gattaca
Alien³
The Rock
Independence Day
Alien
Prometheus


In [104]:
# pickle.dump(df.to_dict(), open('df_dataframe_dict.pkl', 'wb'))

In [105]:
# pickle.dump(similarity, open('similarity.pkl', 'wb'))

In [106]:
movies.groupby('title')['vote_average']